***
# chapter 10. DDL 
(Data Definition Language)
***
## keyword
* create
* alter 

## 10-1. CREATE (테이블 생성)


In [2]:
import pandas as pd
import cx_Oracle
xedb = cx_Oracle.connect('hr/hr@localhost/xe')
cur = xedb.cursor()

In [3]:
# emp 테이블 생성
cur.execute("""CREATE TABLE emp
            AS SELECT employee_id, last_name, salary, department_id 
            FROM employees WHERE 1=2""")

# emp 테이블 구조(컬럼)확인
cur.execute('select * from emp')
for c in cur.description:
    print(c)

('EMPLOYEE_ID', <class 'cx_Oracle.NUMBER'>, 7, None, 6, 0, 1)
('LAST_NAME', <class 'cx_Oracle.STRING'>, 25, 50, None, None, 0)
('SALARY', <class 'cx_Oracle.NUMBER'>, 12, None, 8, 2, 1)
('DEPARTMENT_ID', <class 'cx_Oracle.NUMBER'>, 5, None, 4, 0, 1)


In [4]:
# dept 테이블 생성
cur.execute("""CREATE TABLE dept
            AS SELECT * FROM departments WHERE 1=2""")

# dept 테이블 구조(컬럼) 확인
cur.execute('select * from dept')
for c in cur.description:
    print(c)
    

('DEPARTMENT_ID', <class 'cx_Oracle.NUMBER'>, 5, None, 4, 0, 1)
('DEPARTMENT_NAME', <class 'cx_Oracle.STRING'>, 30, 60, None, None, 0)
('MANAGER_ID', <class 'cx_Oracle.NUMBER'>, 7, None, 6, 0, 1)
('LOCATION_ID', <class 'cx_Oracle.NUMBER'>, 5, None, 4, 0, 1)


## 10-2. ALTER
### < ADD_열 추가>

In [5]:
# job_id 열추가
cur.execute("ALTER TABLE emp ADD (job_id VARCHAR2(9))")

cur.execute('select * from emp')
for c in cur.description:
    print(c)

('EMPLOYEE_ID', <class 'cx_Oracle.NUMBER'>, 7, None, 6, 0, 1)
('LAST_NAME', <class 'cx_Oracle.STRING'>, 25, 50, None, None, 0)
('SALARY', <class 'cx_Oracle.NUMBER'>, 12, None, 8, 2, 1)
('DEPARTMENT_ID', <class 'cx_Oracle.NUMBER'>, 5, None, 4, 0, 1)
('JOB_ID', <class 'cx_Oracle.STRING'>, 9, 18, None, None, 1)


### < MODIFY_열 수정 >
* 데이터가 존재할 경우 데이터타입의 수정은 불가
* 데이터크기 조정 가능

In [6]:
# last_name 데이터크기 조정
cur.execute("ALTER TABLE emp MODIFY (last_name VARCHAR2(30))")

cur.execute('select * from emp')
for c in cur.description:
    print(c)

('EMPLOYEE_ID', <class 'cx_Oracle.NUMBER'>, 7, None, 6, 0, 1)
('LAST_NAME', <class 'cx_Oracle.STRING'>, 30, 60, None, None, 0)
('SALARY', <class 'cx_Oracle.NUMBER'>, 12, None, 8, 2, 1)
('DEPARTMENT_ID', <class 'cx_Oracle.NUMBER'>, 5, None, 4, 0, 1)
('JOB_ID', <class 'cx_Oracle.STRING'>, 9, 18, None, None, 1)


## 10-3. DROP
* DROP
* SET UNUSED

In [7]:
# job_id 컬럼 삭제
cur.execute("ALTER TABLE emp DROP COLUMN job_id")  #LOCK을 걸고 데이터를 삭제

cur.execute('select * from emp')
for c in cur.description:
    print(c)

('EMPLOYEE_ID', <class 'cx_Oracle.NUMBER'>, 7, None, 6, 0, 1)
('LAST_NAME', <class 'cx_Oracle.STRING'>, 30, 60, None, None, 0)
('SALARY', <class 'cx_Oracle.NUMBER'>, 12, None, 8, 2, 1)
('DEPARTMENT_ID', <class 'cx_Oracle.NUMBER'>, 5, None, 4, 0, 1)


### < SET UNUSED >

In [9]:
# salary 컬럼에 LOCK해서 조회되지 않지만, 스토리지에서 삭제된 것은 아님
cur.execute("ALTER TABLE emp SET UNUSED (salary)") 

cur.execute('select * from emp')
for c in cur.description:
    print(c)

('EMPLOYEE_ID', <class 'cx_Oracle.NUMBER'>, 7, None, 6, 0, 1)
('LAST_NAME', <class 'cx_Oracle.STRING'>, 30, 60, None, None, 0)
('DEPARTMENT_ID', <class 'cx_Oracle.NUMBER'>, 5, None, 4, 0, 1)


In [10]:
# unused column 조회
data = pd.read_sql("SELECT * FROM user_unused_col_tabs",xedb)
print(data) 

  TABLE_NAME  COUNT
0        EMP      1


In [11]:
# UNUSED COLUMN 삭제
cur.execute("ALTER TABLE emp DROP UNUSED COLUMNS")

data = pd.read_sql("SELECT * FROM user_unused_col_tabs",xedb)
print(data)        

Empty DataFrame
Columns: [TABLE_NAME, COUNT]
Index: []


# [ Constraint (integrity Constraint) ]
테이블 작성시 constraint를 사용해서 입력하는 자료에 대한 규칙을 정해둔다. 정해진 제약에 따라 데이터가 입력이 되어 유효하지 않은 데이터가 입력되는 것을 방지한다.
#### NOT NULL
필수적으로 입력해야 할 컬럼에 설정 (NULL 불허)

#### UNIQUE
중복값 불허 
즉, 테이블 모든 행에 대해서 유일값이 필히 존재 (NULL허용) 

#### PRIMARY KEY (기본키)
테이블에서 대표되는 컬럼 
NULL, 중복값 모두 불허 (NOT NULL + UNIQUE)
데이터의 특정조건을 검색, 수정 등의 작업을 할 때 기본키로 구분

#### FOREIGN KEY (외래키)
참조하는 테이블 컬럼의 데이터만 허용
(참조하는 테이블은 PRIMARY와 UNIQUE로,  지정된 컬럼만을 FOREIGN으로 지정할 수 있다.)

####  CHECK
주어진 조건에 해당되는 데이터만 허용


## < PRIMANRY KEY >
* 유일한 값만 / NULL x / 중복성 x
* PRIMARY key를 생성하면 UNIQUE index는 자동 생성
* 전체 중 한 건만 찾는, 풀스캔이 아닌 rowid스캔이 효율적 -> 인덱스 생성이 유리

#### dept 테이블 department_id에 PK설정

In [12]:
cur.execute("ALTER TABLE dept ADD CONSTRAINT deptid_pk PRIMARY KEY(department_id)")
# Deptid_pk : 제약조건이름 = index_name

data = pd.read_sql("""SELECT constraint_name, constraint_type, search_condition, index_name, status
                  FROM user_constraints
                  WHERE table_name = 'DEPT'
                  """,xedb)
print(data)

  CONSTRAINT_NAME CONSTRAINT_TYPE               SEARCH_CONDITION INDEX_NAME  \
0     SYS_C008000               C  "DEPARTMENT_NAME" IS NOT NULL       None   
1       DEPTID_PK               P                           None  DEPTID_PK   

    STATUS  
0  ENABLED  
1  ENABLED  


#### emp 테이블 employee_id에 PK설정

In [13]:
cur.execute("ALTER TABLE emp ADD CONSTRAINT empid_pk PRIMARY KEY(employee_id)")

data = pd.read_sql("""SELECT constraint_name, constraint_type, search_condition, index_name, status
                    FROM user_constraints
                    WHERE table_name = 'EMP'
                    """,xedb)
print(data)

  CONSTRAINT_NAME CONSTRAINT_TYPE         SEARCH_CONDITION INDEX_NAME   STATUS
0     SYS_C007999               C  "LAST_NAME" IS NOT NULL       None  ENABLED
1        EMPID_PK               P                     None   EMPID_PK  ENABLED


## < FOREIGN KEY  >
* PK에 있는 키값만 입력받을 수 있다. 
* PK 컬럼 삭제시 FK 컬럼에 그 값이 입력되어있으면 삭제불가
  부모테이블을 삭제하기 위해서는 자식 테이블을 먼저 삭제
(단, FK 선언 때 ON DELETE CASCADE나 ON DELETE SET NULL옵션을 사용한 경우에는 삭제가능)

#### emp테이블 department_id에 FK설정 
##### 1. 옵션: DELETE CASCADE
* DELETE CASADE : 상위 테이블의 행이 삭제될 때 하위테이블의 종속행을 삭제
(기본값:pk에 종속된 row가 있으면 삭제 불허)

In [14]:
cur.execute("""ALTER TABLE emp ADD CONSTRAINT emp_dept_id_fk FOREIGN KEY (department_id)
                                              REFERENCES dept(department_id) ON DELETE CASCADE """)

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition, r_constraint_name, delete_rule, status
                   FROM user_constraints
                   WHERE table_name = 'EMP'
                   """,xedb)
print(data)
# delete_rule 확인

  CONSTRAINT_NAME CONSTRAINT_TYPE         SEARCH_CONDITION R_CONSTRAINT_NAME  \
0     SYS_C007999               C  "LAST_NAME" IS NOT NULL              None   
1        EMPID_PK               P                     None              None   
2  EMP_DEPT_ID_FK               R                     None         DEPTID_PK   

  DELETE_RULE   STATUS  
0        None  ENABLED  
1        None  ENABLED  
2     CASCADE  ENABLED  


##### 2. 옵션: DELETE SET NULL  
* DELETE SET NULL (옵션) 
종속 FOREIGN KEY값을 NULL로 변환
: PK row를 삭제할 때 FK가 걸려있는 CHILD row의 필드값만 null로 업데이트 


In [15]:
 # 이전 FK ( DELETE CASCADE 삭제)
cur.execute("ALTER TABLE emp DROP CONSTRAINT emp_dept_id_fk")
cur.execute("""ALTER TABLE emp ADD CONSTRAINT emp_dept_id_fk  FOREIGN KEY (department_id)
                                              REFERENCES dept(department_id) ON DELETE SET NULL""")

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition, r_constraint_name,delete_rule, status
                   FROM user_constraints
                   WHERE table_name = 'EMP'
                   """,xedb)
print(data)
# delete_rule 확인

  CONSTRAINT_NAME CONSTRAINT_TYPE         SEARCH_CONDITION R_CONSTRAINT_NAME  \
0     SYS_C007999               C  "LAST_NAME" IS NOT NULL              None   
1        EMPID_PK               P                     None              None   
2  EMP_DEPT_ID_FK               R                     None         DEPTID_PK   

  DELETE_RULE   STATUS  
0        None  ENABLED  
1        None  ENABLED  
2    SET NULL  ENABLED  


#### 3. 기본값

In [16]:
# 이전 FK 삭제
cur.execute("ALTER TABLE emp DROP CONSTRAINT emp_dept_id_fk")

# 기본값
cur.execute("""ALTER TABLE emp  ADD CONSTRAINT emp_dept_id_fk FOREIGN KEY(department_id)
                                              REFERENCES dept(department_id)""")

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition, r_constraint_name,delete_rule, status
                   FROM user_constraints
                   WHERE table_name = 'EMP'
                   """,xedb)
print(data)
# delete_rule 확인 (NO ACTION)

  CONSTRAINT_NAME CONSTRAINT_TYPE         SEARCH_CONDITION R_CONSTRAINT_NAME  \
0     SYS_C007999               C  "LAST_NAME" IS NOT NULL              None   
1        EMPID_PK               P                     None              None   
2  EMP_DEPT_ID_FK               R                     None         DEPTID_PK   

  DELETE_RULE   STATUS  
0        None  ENABLED  
1        None  ENABLED  
2   NO ACTION  ENABLED  


## < CONSTRAINT 수정 >
* not null제약조건만 수정가능 (not null-> null로 수정) 

In [17]:
# 수정 후 (emp테이블의 last_name 조건 수정)
cur.execute("ALTER TABLE emp MODIFY (last_name VARCHAR2(30) null)")

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition, r_constraint_name,delete_rule, status
                   FROM user_constraints
                   WHERE table_name = 'EMP'
                   """,xedb)
print(data)
# SEARCH_CONDITION 확인

  CONSTRAINT_NAME CONSTRAINT_TYPE SEARCH_CONDITION R_CONSTRAINT_NAME  \
0        EMPID_PK               P             None              None   
1  EMP_DEPT_ID_FK               R             None         DEPTID_PK   

  DELETE_RULE   STATUS  
0        None  ENABLED  
1   NO ACTION  ENABLED  


In [18]:
cur.execute('select * from emp')
for c in cur.description:
    print(c)

('EMPLOYEE_ID', <class 'cx_Oracle.NUMBER'>, 7, None, 6, 0, 0)
('LAST_NAME', <class 'cx_Oracle.STRING'>, 30, 60, None, None, 1)
('DEPARTMENT_ID', <class 'cx_Oracle.NUMBER'>, 5, None, 4, 0, 1)


## < NOT NULL >

In [19]:
# null로 수정했던 emp테이블의 last_name 조건을 다시 not null로
cur.execute("ALTER TABLE emp MODIFY (last_name VARCHAR2(30) constraint emp_name_nn not null)")

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition, r_constraint_name,delete_rule, status
                   FROM user_constraints
                   WHERE table_name = 'EMP'
                   """,xedb)
print(data)

  CONSTRAINT_NAME CONSTRAINT_TYPE         SEARCH_CONDITION R_CONSTRAINT_NAME  \
0     EMP_NAME_NN               C  "LAST_NAME" IS NOT NULL              None   
1        EMPID_PK               P                     None              None   
2  EMP_DEPT_ID_FK               R                     None         DEPTID_PK   

  DELETE_RULE   STATUS  
0        None  ENABLED  
1        None  ENABLED  
2   NO ACTION  ENABLED  


In [20]:
cur.execute('select * from emp')
for c in cur.description:
    print(c)

('EMPLOYEE_ID', <class 'cx_Oracle.NUMBER'>, 7, None, 6, 0, 0)
('LAST_NAME', <class 'cx_Oracle.STRING'>, 30, 60, None, None, 0)
('DEPARTMENT_ID', <class 'cx_Oracle.NUMBER'>, 5, None, 4, 0, 1)


## < DISABLE _ 제약 조건 비활성화 >
* CREATE TABLE, ALTER TABLE에 DISABLE 사용가능
* UNIQUE 또는 PRIMARY KEY를 비활성화하면 UNIQUE 인덱스 제거

In [21]:
cur.execute("ALTER TABLE emp DISABLE CONSTRAINT empid_pk")

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition, r_constraint_name,delete_rule, status
                   FROM user_constraints
                   WHERE table_name = 'EMP'
                   """,xedb)
print(data)
#STATUS 확인

  CONSTRAINT_NAME CONSTRAINT_TYPE         SEARCH_CONDITION R_CONSTRAINT_NAME  \
0     EMP_NAME_NN               C  "LAST_NAME" IS NOT NULL              None   
1        EMPID_PK               P                     None              None   
2  EMP_DEPT_ID_FK               R                     None         DEPTID_PK   

  DELETE_RULE    STATUS  
0        None   ENABLED  
1        None  DISABLED  
2   NO ACTION   ENABLED  


## < ENABLE _ 제약 조건 활성화 >
* 제약 조건을 활성화하면 해당 제약 조건이 테이블의 모든 데이터에 적용 
* UNIQUE KEY 또는 PRIMARY KEY를 활성화하면 UNIQUE 또는 PRIMARY KEY 인덱스가 자동 생성
* CREATE TABLE과 ALTER TABLE에 ENABLE 사용가능
* PK먼저 ENABLE, 그 다음 FK ENABLE(종속관계) 그러므로 CASCADE 없다

In [22]:
cur.execute("ALTER TABLE emp ENABLE CONSTRAINT empid_pk")

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition, r_constraint_name,delete_rule, status
                   FROM user_constraints
                   WHERE table_name = 'EMP'
                   """,xedb)
print(data)

  CONSTRAINT_NAME CONSTRAINT_TYPE         SEARCH_CONDITION R_CONSTRAINT_NAME  \
0     EMP_NAME_NN               C  "LAST_NAME" IS NOT NULL              None   
1        EMPID_PK               P                     None              None   
2  EMP_DEPT_ID_FK               R                     None         DEPTID_PK   

  DELETE_RULE   STATUS  
0        None  ENABLED  
1        None  ENABLED  
2   NO ACTION  ENABLED  


In [23]:
data = pd.read_sql("""SELECT index_name, column_name
                   FROM user_ind_columns
                   WHERE table_name = 'EMP'
                   """,xedb)
print(data)

  INDEX_NAME  COLUMN_NAME
0   EMPID_PK  EMPLOYEE_ID


# [ PRACTICE 1 ]
#### 1. 테이블 생성

In [24]:
cur.execute("CREATE TABLE NEW_EMP (id NUMBER(6), name VARCHAR2(20))")

#### 2. UNIQUE 인덱스 생성

In [25]:
cur.execute("CREATE UNIQUE INDEX new_emp_id_idx ON new_emp(ID)")

# 인덱스정보확인
data = pd.read_sql("""SELECT index_name, column_name 
                   FROM user_ind_columns
                   WHERE table_name = 'NEW_EMP'
                   """,xedb)
print(data)

       INDEX_NAME COLUMN_NAME
0  NEW_EMP_ID_IDX          ID


#### 3. PRIMARY KEY 설정

In [26]:
cur.execute("ALTER TABLE new_emp ADD CONSTRAINT new_em_pk PRIMARY KEY (ID) USING INDEX new_emp_id_idx")
# USING INDEX : 인덱스 생성불가 하도록 요청 (이미 인덱스가 생성이 되어있는 상태인데, PK하면서 인덱시 반복 생성되므로)

data = pd.read_sql("""SELECT constraint_name, constraint_type, index_name
                   FROM user_constraints
                   WHERE table_name = 'NEW_EMP'
                   """,xedb)
print(data)

  CONSTRAINT_NAME CONSTRAINT_TYPE      INDEX_NAME
0       NEW_EM_PK               P  NEW_EMP_ID_IDX


#### 4. 인덱스 삭제

In [27]:
cur.execute("DROP INDEX new_emp_id_idx")
# 인덱스 삭제 불가. PK 삭제 후 가능

DatabaseError: ORA-02429: cannot drop index used for enforcement of unique/primary key

In [28]:
# PK삭제
cur.execute("ALTER TABLE new_emp DROP PRIMARY KEY ")

data = pd.read_sql("""SELECT constraint_name, constraint_type, index_name
                   FROM user_constraints
                   WHERE table_name = 'NEW_EMP'
                   """,xedb)
print(data)

Empty DataFrame
Columns: [CONSTRAINT_NAME, CONSTRAINT_TYPE, INDEX_NAME]
Index: []


In [29]:
data = pd.read_sql("""SELECT index_name, column_name
                   FROM user_ind_columns 
                   WHERE table_name = 'NEW_EMP'
                   """,xedb)
print(data)

       INDEX_NAME COLUMN_NAME
0  NEW_EMP_ID_IDX          ID


In [30]:
# 인덱스 삭제
cur.execute("DROP INDEX new_emp_id_idx")

data = pd.read_sql("""SELECT index_name, column_name
                   FROM user_ind_columns
                   WHERE table_name = 'NEW_EMP'
                   """,xedb)
print(data)

Empty DataFrame
Columns: [INDEX_NAME, COLUMN_NAME]
Index: []


# [ PRACTICE 2 ]

In [4]:
# 테이블 생성
cur.execute("""CREATE TABLE temp_emp
            AS SELECT * FROM employees""")

data = pd.read_sql("""SELECT index_name, column_name
                   FROM user_ind_columns
                   WHERE table_name = 'TEMP_EMP'
                   """,xedb)
print(data)

Empty DataFrame
Columns: [INDEX_NAME, COLUMN_NAME]
Index: []


#### PK와 UNIQUE제약조건은 UNIQUE INDEX 자동생성

In [5]:
# PK 설정 및 인덱스 확인
cur.execute("ALTER TABLE temp_emp ADD CONSTRAINT temp_emp_id_pk PRIMARY KEY(employee_id)")

data = pd.read_sql("""SELECT index_name, column_name
                   FROM user_ind_columns
                   WHERE table_name = 'TEMP_EMP'
                   """,xedb)
print(data)

       INDEX_NAME  COLUMN_NAME
0  TEMP_EMP_ID_PK  EMPLOYEE_ID


## < DROP, PURGE > 
* 테이블 삭제
* 테이블에 걸려있는 인덱스, 제약조건도 동시삭제

In [6]:
cur.execute("DROP TABLE temp_emp")
# DROP TABLE temp_emp PURGE; 영구삭제

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition
                   FROM user_constraints
                   WHERE table_name = 'TEMP_EMP'
                   """,xedb)
print(data)

Empty DataFrame
Columns: [CONSTRAINT_NAME, CONSTRAINT_TYPE, SEARCH_CONDITION]
Index: []


In [7]:
data = pd.read_sql("""SELECT index_name, column_name
                   FROM user_ind_columns
                   WHERE table_name = 'TEMP_EMP'
                   """,xedb)
print(data)

Empty DataFrame
Columns: [INDEX_NAME, COLUMN_NAME]
Index: []


In [8]:
# 휴지통 보기 (테이블 삭제 후 휴지통에 RENAME되어있다)

data = pd.read_sql("""SELECT OBJECT_NAME, ORIGINAL_NAME, TYPE 
                   FROM RECYCLEBIN
                   """,xedb)
print(data)  

                      OBJECT_NAME   ORIGINAL_NAME   TYPE
0  BIN$ScSPTmkwTOyU5vQrYrG0vw==$0  TEMP_EMP_ID_PK  INDEX
1  BIN$2HRjszLiSNuzDB0IIS+puA==$0        TEMP_EMP  TABLE


In [10]:
# RENAME된 이름으로 조회

data = pd.read_sql("""SELECT count(*) 
                    FROM "BIN$2HRjszLiSNuzDB0IIS+puA==$0"
                    """,xedb)
print(data)

   COUNT(*)
0       107


## < FLASHBACK_ 복원작업 >
* 인덱스, 제약조건도 함께 복원
* 이름복원 없이 bin으로 표기되기 때문에 이름수정필요

In [11]:
cur.execute("FLASHBACK TABLE temp_emp TO BEFORE DROP")

data = pd.read_sql("SELECT count(*) FROM temp_emp",xedb)
print(data)

   COUNT(*)
0       107


In [12]:
# CONSTRAINT 확인

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition
                   FROM user_constraints
                   WHERE table_name = 'TEMP_EMP'
                   """,xedb)
print(data)

                  CONSTRAINT_NAME CONSTRAINT_TYPE         SEARCH_CONDITION
0  BIN$NRYlvuczShOHFFvsSn16Aw==$0               C  "LAST_NAME" IS NOT NULL
1  BIN$KFpJCZpWQwuuUO3VK1Wx0Q==$0               C      "EMAIL" IS NOT NULL
2  BIN$DTOUxrU0RGqLX3DJ9kQotA==$0               C  "HIRE_DATE" IS NOT NULL
3  BIN$jkJmpGM+QKyCccUuCXjlXQ==$0               C     "JOB_ID" IS NOT NULL
4  BIN$b38OVwgoQ0W9gSvuSvIaLw==$0               P                     None


In [13]:
# INDEX 확인

data = pd.read_sql("""SELECT index_name, column_name
                   FROM user_ind_columns
                   WHERE table_name = 'TEMP_EMP'
                   """,xedb)
print(data)

                       INDEX_NAME  COLUMN_NAME
0  BIN$ScSPTmkwTOyU5vQrYrG0vw==$0  EMPLOYEE_ID


## < RENAME _이름 수정 >
- CONSTRAINT 이름수정

In [18]:
cur.execute("""ALTER TABLE temp_emp RENAME CONSTRAINT "BIN$b38OVwgoQ0W9gSvuSvIaLw==$0" TO TEMP_EMP_ID_PK""")

data = pd.read_sql("""SELECT constraint_name, constraint_type,search_condition
                   FROM user_constraints
                   WHERE table_name = 'TEMP_EMP'
                   """,xedb)
print(data)

                  CONSTRAINT_NAME CONSTRAINT_TYPE         SEARCH_CONDITION
0  BIN$NRYlvuczShOHFFvsSn16Aw==$0               C  "LAST_NAME" IS NOT NULL
1  BIN$KFpJCZpWQwuuUO3VK1Wx0Q==$0               C      "EMAIL" IS NOT NULL
2  BIN$DTOUxrU0RGqLX3DJ9kQotA==$0               C  "HIRE_DATE" IS NOT NULL
3  BIN$jkJmpGM+QKyCccUuCXjlXQ==$0               C     "JOB_ID" IS NOT NULL
4                  TEMP_EMP_ID_PK               P                     None


- INDEX 이름 수정

In [19]:
cur.execute("""ALTER INDEX "BIN$ScSPTmkwTOyU5vQrYrG0vw==$0" RENAME TO TEMP_EMP_ID_PK""")

data = pd.read_sql("""SELECT index_name, column_name
                   FROM user_ind_columns
                   WHERE table_name = 'TEMP_EMP'
                   """,xedb)
print(data)

       INDEX_NAME  COLUMN_NAME
0  TEMP_EMP_ID_PK  EMPLOYEE_ID


In [ ]:
cur.close()
xedb.close()